Notebook to test OTP configuration.
Written by Vivek
Started July 7th, 2016

We need to test the different parameters like bike and walk, etc.  Build functions that test numerous locations, settings, and generate times.

Current Todos:

TODO: Geocode Addresses into GPS using Google

TODO: Push resulting data into pandas dataframe

In [1]:
import requests
import json
import pandas as pd
import numpy as np

Sample Api Call:
http://vta2.amigocloud.com/api/otp/plan?date=2016-07-07&time=23%3A0&mode=WALK%2CBUSISH%2CTRAINISH&fromPlace=37.389268%2C-121.879746&toPlace=37.4031595%2C-121.9865994&numItineraries=3&maxWalkDistance=5000&bikeSpeed=10&walkReluctance=10&maxTransfers=5&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE&arriveBy=false

In [2]:
def query_string(start_lat, start_long, dest_lat, dest_long, visual = False):    
    base = 'http://vta2.amigocloud.com'
    api = '/api/otp/plan?'
    date = 'date=2016-07-07'
    time = '&time=23%3A0'
    mode = '&mode=WALK%2CBUSISH%2CTRAINISH'
    start = '&fromPlace=' + str(start_lat) + '%2C' + str(start_long)
    #start = '&fromPlace=37.389268%2C-121.879746'
    dest = '&toPlace=' + str(dest_lat) + '%2C' + str(dest_long)
    #dest = '&toPlace=37.4031595%2C-121.9865994'
    num_iter = '&numItineraries=3'
    walk = '&maxWalkDistance=5000&'
    bike = 'bikeSpeed=10'
    walk_reluc = '&walkReluctance=10'
    max_tran = '&maxTransfers=5'
    triangle = '&triangleSlopeFactor=0.333&triangleSafetyFactor=0.333&triangleTimeFactor=0.334&optimize=TRIANGLE'
    arriveby = '&arriveBy=false'
    if visual == True:
        api = '/planner?'
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluc + max_tran + triangle + arriveby
    else:
        query = base + api + date + time + mode + start + dest + num_iter + walk + bike + walk_reluc + max_tran + triangle + arriveby

    return query

In [3]:
rand_starts=pd.read_csv('Rand_Starts.csv')
rand_dests=pd.read_csv('Rand_Dests.csv')

In [4]:
df = pd.DataFrame();
df['start_lat'] = rand_starts['Lat']
df['start_long'] = rand_starts['Long']
df['dest_lat'] = rand_dests['Lat']
df['dest_long'] = rand_dests['Long']

In [5]:
df.head()

,start_lat,start_long,dest_lat,dest_long
0,37.538234,-121.991302,37.356750,-122.039860
1,37.126105,-121.672784,37.280022,-121.907950
2,37.381230,-121.859742,37.356277,-122.058223
3,37.385547,-121.859570,37.415452,-122.019959
4,37.293663,-121.812420,37.257288,-121.817952


In [6]:
row = 0
times=[]
results = []
visuals = []
queries = []
for each in df.iterrows():
    query = query_string(start_lat = df['start_lat'][row], start_long = df['start_long'][row], dest_lat = df['dest_lat'][row], dest_long=df['dest_long'][row])
    queries.append(query)
    visuals.append(query_string(start_lat = df['start_lat'][row], start_long = df['start_long'][row], dest_lat = df['dest_lat'][row], dest_long=df['dest_long'][row], visual = True))
    r = requests.get(query)
    query_json = json.loads(r.text)
    results.append(str(query_json))
    one_set_of_times= query_json['debugOutput']['pathTimes']
    times.append(one_set_of_times)
    row = row + 1

In [7]:
df['times'] = times
df['visual'] = visuals
df['results'] = results

In [8]:
trips = []
lowest = []
for each in df['times'][0:50]:
    trips.append(len(each))
    if len(each) != 0:
        lowest.append(each[0])
    else:
        lowest.append(999)
df['trips']=trips
df['lowest']=lowest

In [9]:
df

,start_lat,start_long,dest_lat,dest_long,times,visual,results,trips,lowest
0,37.538234,-121.991302,37.356750,-122.039860,"[782, 594]",http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",2,782
1,37.126105,-121.672784,37.280022,-121.907950,"[13, 11, 41]",http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",3,13
2,37.381230,-121.859742,37.356277,-122.058223,"[304, 47, 58]",http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",3,304
3,37.385547,-121.859570,37.415452,-122.019959,[241],http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",1,241
4,37.293663,-121.812420,37.257288,-121.817952,"[33, 28, 31]",http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",3,33
5,37.294797,-121.869427,37.439579,-121.918965,[],http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",0,999
6,37.390847,-122.005821,37.407480,-122.005678,"[11, 15, 694]",http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",3,11
7,37.285324,-122.045775,37.360400,-121.905028,"[302, 440]",http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",2,302
8,37.131394,-121.668520,37.246916,-121.924915,"[20, 40, 87]",http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",3,20
9,37.251716,-121.906794,37.274348,-121.928639,"[17, 27, 38]",http://vta2.amigocloud.com/planner?date=2016-0...,"{'requestParameters': {'arriveBy': 'false', 't...",3,17


In [10]:
df.to_csv('paths.csv')

In [ ]:
'http://docs.python-guide.org/en/latest/scenarios/json/'

In [ ]:
query_viz